In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
pd.options.display.float_format ='{:,.2f}'.format

В курсовой работе будут исследовано влияние на уровень самоубийств в регионах Российской Федерации различных социально-демографических факторов: уровня рождаемости, уровня общей смертности, уровня убийств и смертности от иных внешних причин (ДТП, пожары и т. п.). 

Нулевая гипотеза: связь между вышеобозначенными явлениями отсутствует.

Альтернативная гипотеза: имеет место прямая связь между сметрностью от разных факторов и самоубийствами и обратная связь между рождаемостью и самоубийствами (т. е. чем выше рождаемость в регионе, тем ниже уровень самоубийств).

В качестве основы взят датасет с сайта Росстата https://rosstat.gov.ru/opendata/7708234640-7708234640-dataset2021

Из него предварительно сделана выдержка в формате pickle (полный датасет занимает более 3 Гб!), которая приложена к работе.

In [2]:
df = pd.read_pickle('demographics.pickle')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1524559 entries, 2751884 to 6156274
Data columns (total 23 columns):
 #   Column                 Non-Null Count    Dtype   
---  ------                 --------------    -----   
 0   object_name            1524559 non-null  category
 1   year                   1524559 non-null  int16   
 2   indicator_name         1524559 non-null  category
 3   original_unit          1524559 non-null  category
 4   area_type              1524559 non-null  category
 5   age_group              1490296 non-null  category
 6   year_of_birth          0 non-null        float32 
 7   sex                    1515379 non-null  category
 8   month                  0 non-null        category
 9   birth_order            0 non-null        category
 10  mother_marital_status  0 non-null        category
 11  death_cause            1404599 non-null  category
 12  bride_age              0 non-null        category
 13  groom_age              0 non-null        category
 

In [3]:
df.columns.to_list()

['object_name',
 'year',
 'indicator_name',
 'original_unit',
 'area_type',
 'age_group',
 'year_of_birth',
 'sex',
 'month',
 'birth_order',
 'mother_marital_status',
 'death_cause',
 'bride_age',
 'groom_age',
 'migration_direction',
 'country',
 'region',
 'territory_type',
 'registration_type',
 'citizenship',
 'indicator_value',
 'original_accuracy',
 'notes']

In [4]:
df = df[['object_name','year','indicator_name','original_unit','area_type','age_group','sex','death_cause','indicator_value']]

In [5]:
df[['object_name','indicator_name','original_unit','area_type','age_group','sex','death_cause']] = df[['object_name','indicator_name','original_unit','area_type','age_group','sex','death_cause']].astype('str')

In [6]:
df['indicator_name'] = df['indicator_name'] + ', ' + df['original_unit']
df = df.drop('original_unit', axis=1)

In [7]:
df.sample(5)

,object_name,year,indicator_name,area_type,age_group,sex,death_cause,indicator_value
5107869,Оренбургская область,2008,"Число умерших по причинам смерти, человек",сельское население,все возраста,мужчины и женщины,случайное отравление (воздействие) алкоголем,149.00
4812663,Амурская область,2009,"Число умерших по причинам смерти, человек",сельское население,0–17 лет,мужчины,внешние причины смерти,20.00
4633552,Республика Коми,1997,"Число умерших по причинам смерти, человек",городское население,все возраста,мужчины и женщины,новообразования,"1,164.00"
4643548,Белгородская область,2011,"Число умерших по причинам смерти, человек",городское население,все возраста,мужчины и женщины,"убийство (нападение, насилие)",56.00
4054660,Северо-Кавказский федеральный округ,2018,"Число умерших по причинам смерти, человек",городское и сельское население,0–17 лет,мужчины и женщины,"врожденные аномалии (пороки развития), деформа...",292.00


Первоначальный датасет содержит много избыточной информации и не приспособлен для анализа. Так, часть признаков содержится в колонках, а часть (например, причины смерти) -  в строках. Данные нужно привести в "плоский", удобочитаемый формат. Для этого разобьем исходный датасет на два сета: общая информация и информация по причинам смерти. 

In [8]:
general = df.query('indicator_name != "Число умерших по причинам смерти, человек"')
death = df.query('indicator_name == "Число умерших по причинам смерти, человек"')

In [9]:
general.query('object_name == "Санкт-Петербург"')

,object_name,year,indicator_name,area_type,age_group,sex,death_cause,indicator_value
2753984,Санкт-Петербург,1990,Численность постоянного населения в среднем за...,городское и сельское население,nan,nan,nan,"5,004,955.00"
2753985,Санкт-Петербург,1991,Численность постоянного населения в среднем за...,городское и сельское население,nan,nan,nan,"4,996,936.00"
2753986,Санкт-Петербург,1992,Численность постоянного населения в среднем за...,городское и сельское население,nan,nan,nan,"4,964,638.00"
2753987,Санкт-Петербург,1993,Численность постоянного населения в среднем за...,городское и сельское население,nan,nan,nan,"4,912,231.00"
2753988,Санкт-Петербург,1994,Численность постоянного населения в среднем за...,городское и сельское население,nan,nan,nan,"4,863,488.00"
...,...,...,...,...,...,...,...,...
3712740,Санкт-Петербург,2018,"Число умерших, человек",сельское население,все возраста,мужчины и женщины,nan,0.00
3712741,Санкт-Петербург,2019,"Число умерших, человек",сельское население,все возраста,мужчины и женщины,nan,0.00
6153767,Санкт-Петербург,1993,"Число умерших, человек",городское население,до 1 года,женщины,nan,281.00
6153850,Санкт-Петербург,1993,"Число умерших, человек",городское население,в трудоспособном возрасте,женщины,nan,"5,162.00"


Из датасета нужно убрать лишнюю информацию из полей age_group, sex, area_type, а данные из поля indicator_name перевести в колонки. Кроме того, по ряду показателей за 90-е годы отсутствует итоговая статистика, а есть только статистика в разбивке по полу (т. е. отдельно заполнены значения "мужчины", "женщины", а значение "мужчины и женщины" - нет)

In [10]:
general = general.query('area_type == "городское и сельское население" and sex != "мужчины и женщины"').drop(['death_cause'], axis=1)
general = general.query('age_group == "все возраста" or age_group == "nan"')
general = general.groupby(['object_name', 'year', 'indicator_name', 'age_group'],as_index=False).indicator_value.sum().drop('age_group', axis=1)
general.query('object_name == "Санкт-Петербург"').head()

,object_name,year,indicator_name,indicator_value
6041,Санкт-Петербург,1990,Численность постоянного населения в среднем за...,"5,004,955.00"
6042,Санкт-Петербург,1990,"Число родившихся живыми, человек","54,322.00"
6043,Санкт-Петербург,1990,"Число умерших, человек","61,534.00"
6044,Санкт-Петербург,1991,Численность постоянного населения в среднем за...,"4,996,936.00"
6045,Санкт-Петербург,1991,"Число родившихся живыми, человек","46,570.00"


In [11]:
general = general.groupby(['object_name', 'year',  'indicator_name'])['indicator_value'].aggregate('first').unstack().reset_index().rename_axis('index', axis=1)
general.query('object_name == "Санкт-Петербург"').head()

index,object_name,year,"Численность постоянного населения в среднем за год, человек","Число родившихся живыми, человек","Число умерших, человек"
2134,Санкт-Петербург,1990,"5,004,955.00","54,322.00","61,534.00"
2135,Санкт-Петербург,1991,"4,996,936.00","46,570.00","62,715.00"
2136,Санкт-Петербург,1992,"4,964,638.00","37,796.00","67,181.00"
2137,Санкт-Петербург,1993,"4,912,231.00","32,336.00","85,687.00"
2138,Санкт-Петербург,1994,"4,863,488.00","34,563.00","83,647.00"


В таком виде данные пригодны для анализа

In [12]:
death = death.query('area_type == "городское и сельское население" and age_group == "все возраста" and sex != "мужчины и женщины"' )
death = death.drop(['indicator_name', 'area_type', 'age_group'], axis=1)
death = death.groupby(['object_name', 'year', 'death_cause'],as_index=False).indicator_value.sum()
death.query('object_name == "Санкт-Петербург"').head()

,object_name,year,death_cause,indicator_value
31782,Санкт-Петербург,1990,"болезни и состояния, возникающие в перинатальн...",544.00
31783,Санкт-Петербург,1990,болезни органов дыхания,"1,518.00"
31784,Санкт-Петербург,1990,болезни органов пищеварения,"1,832.00"
31785,Санкт-Петербург,1990,болезни системы кровообращения,"35,701.00"
31786,Санкт-Петербург,1990,внешние причины смерти,"5,719.00"


In [13]:
death.death_cause.value_counts()

болезни органов пищеварения                                                                                                                  2823
болезни системы кровообращения                                                                                                               2823
внешние причины смерти                                                                                                                       2823
врожденные аномалии (пороки развития), деформации и хромосомные нарушения                                                                    2823
злокачественные новообразования                                                                                                              2823
ишемическая болезнь сердца                                                                                                                   2823
некоторые инфекционные и паразитарные болезни                                                                               

Выберем из причин смерти только интересующие

In [14]:
death.death_cause = death.death_cause.replace('внешние причины смерти|повреждения с неопределенными намерениями', 'иные_внешние_причины', regex=True)
death.death_cause = death.death_cause.replace('транспортные несчастные случаи всех видов|ДТП', 'ДТП', regex=True)
lst = ['самоубийства', 'убийство (нападение, насилие)', 'ДТП', 'иные_внешние_причины']
death = death[death.death_cause.isin(lst)]

In [15]:
death = death.groupby(['object_name', 'year',  'death_cause'])['indicator_value'].aggregate('first').unstack().reset_index().rename_axis('index', axis=1)
death.query('object_name == "Санкт-Петербург"').head()

index,object_name,year,ДТП,иные_внешние_причины,самоубийства,"убийство (нападение, насилие)"
1980,Санкт-Петербург,1990,"1,092.00","5,719.00",924.00,411.00
1981,Санкт-Петербург,1991,"1,249.00","6,394.00","1,002.00",537.00
1982,Санкт-Петербург,1992,"1,238.00","8,997.00","1,115.00",907.00
1983,Санкт-Петербург,1993,"1,206.00","12,628.00","1,182.00","1,365.00"
1984,Санкт-Петербург,1994,883.00,"12,270.00","1,120.00","1,394.00"


Данные пригодны для анализа. Теперь объединим обе таблицы

In [16]:
df = general.merge(death, on = ['object_name', 'year'], how= 'left')
df.query('object_name == "Санкт-Петербург"').head()

index,object_name,year,"Численность постоянного населения в среднем за год, человек","Число родившихся живыми, человек","Число умерших, человек",ДТП,иные_внешние_причины,самоубийства,"убийство (нападение, насилие)"
2134,Санкт-Петербург,1990,"5,004,955.00","54,322.00","61,534.00","1,092.00","5,719.00",924.00,411.00
2135,Санкт-Петербург,1991,"4,996,936.00","46,570.00","62,715.00","1,249.00","6,394.00","1,002.00",537.00
2136,Санкт-Петербург,1992,"4,964,638.00","37,796.00","67,181.00","1,238.00","8,997.00","1,115.00",907.00
2137,Санкт-Петербург,1993,"4,912,231.00","32,336.00","85,687.00","1,206.00","12,628.00","1,182.00","1,365.00"
2138,Санкт-Петербург,1994,"4,863,488.00","34,563.00","83,647.00",883.00,"12,270.00","1,120.00","1,394.00"


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3117 entries, 0 to 3116
Data columns (total 9 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   object_name                                                  3117 non-null   object 
 1   year                                                         3117 non-null   int64  
 2   Численность постоянного населения в среднем за год, человек  3060 non-null   float32
 3   Число родившихся живыми, человек                             2787 non-null   float32
 4   Число умерших, человек                                       2823 non-null   float32
 5   ДТП                                                          2823 non-null   float32
 6   иные_внешние_причины                                         2823 non-null   float32
 7   самоубийства                                                 2823 non-null   fl

Таблица приобрела нужный формат, онако часть данных в ней не заполнена. Это возникло из-за путаницы с наименованиями регионов в исходных данных (за период с 92 г. многие регионы переименовывались, объединялись, укрупнялись, упразднялись и снова возникали, появлялись из ниоткуда (Крым) и т. п.) Возьмем только те записи, где есть статистика по самоубийствам.

In [18]:
df = df.query('самоубийства  == самоубийства')

Посчитаем нужные показатели в относительном выражении на 1000 чел. населения

In [19]:
 
df[['Число родившихся живыми, человек', 'Число умерших, человек', 'ДТП','иные_внешние_причины', 'самоубийства','убийство (нападение, насилие)']] = 1000*df[['Число родившихся живыми, человек', 'Число умерших, человек', 'ДТП','иные_внешние_причины', 'самоубийства','убийство (нападение, насилие)']].div(df['Численность постоянного населения в среднем за год, человек'], axis=0)
df.columns=['object_name', 'year', 'pop', 'fert_rate', 'death_rate', 'car_acdnts_death_rate', 'other_ext_death_rate', 'suicide_death_rate', 'violence_death_rate']
df.query('object_name == "Санкт-Петербург"').head()

,object_name,year,pop,fert_rate,death_rate,car_acdnts_death_rate,other_ext_death_rate,suicide_death_rate,violence_death_rate
2134,Санкт-Петербург,1990,"5,004,955.00",10.85,12.29,0.22,1.14,0.18,0.08
2135,Санкт-Петербург,1991,"4,996,936.00",9.32,12.55,0.25,1.28,0.20,0.11
2136,Санкт-Петербург,1992,"4,964,638.00",7.61,13.53,0.25,1.81,0.22,0.18
2137,Санкт-Петербург,1993,"4,912,231.00",6.58,17.44,0.25,2.57,0.24,0.28
2138,Санкт-Петербург,1994,"4,863,488.00",7.11,17.20,0.18,2.52,0.23,0.29


В таблице перемешаны данные о регионах, федеральных округах и РФ в целом, это приводит к дублированию данных и искажению результатов. Исключим из таблицы записи, не относящиеся к регионам.

In [20]:
lst = [  'Российская Федерация', 'Дальневосточный федеральный округ','Северо-Западный федеральный округ','Северо-Кавказский федеральный округ','Сибирский федеральный округ', 'Южный федеральный округ', 'Приволжский федеральный округ', 'Центральный федеральный округ', 'Крымский федеральный округ']
df = df[~df.object_name.isin(lst)]

Теперь данные полностью готовы для анализа

In [21]:
df.set_index(['year', 'pop']).corr()

,fert_rate,death_rate,car_acdnts_death_rate,other_ext_death_rate,suicide_death_rate,violence_death_rate
fert_rate,1.00,-0.61,-0.18,-0.20,-0.09,0.04
death_rate,-0.61,1.00,0.28,0.44,0.23,0.13
car_acdnts_death_rate,-0.18,0.28,1.00,0.50,0.39,0.48
other_ext_death_rate,-0.20,0.44,0.50,1.00,0.82,0.83
suicide_death_rate,-0.09,0.23,0.39,0.82,1.00,0.72
violence_death_rate,0.04,0.13,0.48,0.83,0.72,1.00


Выводы:
1) связь между рождаемостью и уровнем самоубийств отсутствует.
2) связь между уровнем общей смертности и уровнем самоубийств незначительна (здесь еще надо делать поправку на то, что одноя явление является составной частью второго, что само по себе создает корреляцию),
3) существует прямая связь средней степени выраженности между уровнем самоубийств и уровнем смертности от ДТП
4) существует значительная прямая связь между уровнем самоубийств и уровнем насильственных смертей (убийства),
5) существует ярко выраженная прямая связь между уровнем самоубийств и уровнем смертей по иным внешним причинам. 

In [22]:
df.query('year == 2019')[['object_name', 'fert_rate', 'suicide_death_rate']].sort_values('fert_rate', ascending=False).head(10)

,object_name,fert_rate,suicide_death_rate
2918,Чеченская Республика,20.31,0.00
1983,Республика Тыва,18.55,0.27
1683,Республика Ингушетия,16.43,0.00
1653,Республика Дагестан,14.84,0.02
1563,Республика Алтай,13.59,0.35
1158,Ненецкий автономный округ,13.30,0.30
1893,Республика Саха (Якутия),13.22,0.23
1623,Республика Бурятия,12.67,0.36
3086,Ямало-Ненецкий автономный округ,12.59,0.12
2643,Тюменская область без автономий,12.56,0.15


In [23]:
df.query('year == 2019')[['object_name', 'fert_rate', 'suicide_death_rate']].sort_values('suicide_death_rate', ascending=False).head(10)

,object_name,fert_rate,suicide_death_rate
1623,Республика Бурятия,12.67,0.36
1563,Республика Алтай,13.59,0.35
419,Еврейская автономная область,10.58,0.33
2013,Республика Хакасия,10.40,0.31
89,Амурская область,10.03,0.30
1158,Ненецкий автономный округ,13.30,0.30
449,Забайкальский край,11.81,0.29
918,Курганская область,9.20,0.29
2673,Удмуртская Республика,9.71,0.27
1983,Республика Тыва,18.55,0.27


На примере последних данных (2019 г.) видно отсутствие связи между рождаемостью и уровнем самоубийств. Наивысшая рождаемость в РФ наблюдается в национальных республиках Сибири и Кавказа, однако при сходной рождаемости уровень самоубийств в Тыве превышает соответствующий уровень Дагестана в 13 раз. В то же время, Тыва и Курганская область имеют близкий уровень самоубийств, хотя рождаемость в Курганской области в 2 раза ниже, чем в Тыве.

In [24]:
df.query('year == 2019')[['object_name', 'violence_death_rate', 'suicide_death_rate']].sort_values('violence_death_rate', ascending=False).head(10)

,object_name,violence_death_rate,suicide_death_rate
1983,Республика Тыва,0.37,0.27
2996,Чукотский автономный округ,0.30,0.18
89,Амурская область,0.18,0.30
419,Еврейская автономная область,0.17,0.33
1038,Магаданская область,0.16,0.16
449,Забайкальский край,0.15,0.29
1623,Республика Бурятия,0.14,0.36
1893,Республика Саха (Якутия),0.13,0.23
1563,Республика Алтай,0.12,0.35
1773,Республика Коми,0.12,0.25


In [25]:
df.query('year == 2019')[['object_name', 'violence_death_rate', 'suicide_death_rate']].sort_values('suicide_death_rate').head(10)

,object_name,violence_death_rate,suicide_death_rate
1683,Республика Ингушетия,0.03,0.00
2918,Чеченская Республика,0.00,0.00
2223,Сахалинская область,0.06,0.00
2793,Хабаровский край,0.03,0.01
179,Астраханская область,0.01,0.01
1923,Республика Северная Осетия – Алания,0.02,0.01
1653,Республика Дагестан,0.02,0.02
1068,Москва,0.02,0.02
1008,Липецкая область,0.03,0.02
2133,Самарская область,0.03,0.03


Неблагополучные в криминальном отношении сибирские регионы (Тыва, Алтай, Бурятия) имеют  высокий уровень самоубийств, в то же время на Кавказе наблюдается обратная картина (хотя достоверность этих данных вызывает сомнения)

In [26]:
df.query('year == 2019')[['object_name', 'other_ext_death_rate', 'suicide_death_rate']].sort_values('other_ext_death_rate', ascending=False).head(10)

,object_name,other_ext_death_rate,suicide_death_rate
1983,Республика Тыва,2.09,0.27
2996,Чукотский автономный округ,1.86,0.18
89,Амурская область,1.80,0.30
449,Забайкальский край,1.75,0.29
1038,Магаданская область,1.72,0.16
2223,Сахалинская область,1.61,0.00
419,Еврейская автономная область,1.59,0.33
1623,Республика Бурятия,1.47,0.36
1563,Республика Алтай,1.41,0.35
2793,Хабаровский край,1.39,0.01


Высокая связь между самоубийствами и смертью по иным внешним причинам выглядит странно, особенно если вспомнить, что основную часть этих иных причин занимает смерть по невыясненным внешним причинам. 

Особенно интересны показатели Сахалинской области и Хабаровского края, имеющих практически нулевую статистику по самоубийствам на фоне соседей со сходной социально-экономической и культурной ситуацией (Бурятия, Магаданская область, Чукотка).

Учитывая специфику вопроса, возможно предположить, что в этих и других регионах имеет место "переписывание" самоубийств на другие причины, прежде всего на смерть по невыясненным внешним причинам

Общие выводы:
1) связи между уровнем рождаемости и уровнем самоубийств в регионах РФ не выявлено,

2) выявлена выраженная прямая зависимость между уровнем самоубийств и уровнем убийств,

3) достоверность данных по самоубийствам в ряде регионах, прежде всего на Дальнем Востоке, вызывает сомнения